In [ ]:
# Instalujemy biblioteke aby połączyć sklearn i pytorch
!pip install skorch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report, ConfusionMatrixDisplay
import torch
import torch.nn as nn
import torch.optim as optim
from skorch import NeuralNetClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Ładowaneie danych
df = pd.read_csv('./winequality-red.csv')

In [ ]:
# Odzielamy target i funkcje
target = pd.DataFrame(df['quality'])
df.drop('quality', axis=1, inplace=True)

In [ ]:
# Normalizacja cech
scaler = MinMaxScaler()
X_normalized = scaler.fit_transform(target)

In [ ]:
# Sprawdzamy pierwsze wiersze
print(df.head())
print(target.head())

In [ ]:
# Przygotowanie do hisotgramu
df_hist = df.copy()
df_hist['quality'] = target

# Tworzenie subplots - ustalanie liczby wierszy i kolumn na podstawie liczby kolumn w DataFrame
num_features = df_hist.shape[1]
nrows = (num_features + 2) // 3  # Zakładamy 3 kolumny na wiersz

fig, ax = plt.subplots(nrows=nrows, ncols=3, figsize=(20, 15))
ax = ax.flatten()  # Spłaszczanie tablicy osi dla łatwiejszej iteracji

# Iteracja po wszystkich kolumnach i tworzenie histogramów
for i, col in enumerate(df_hist.columns):
    ax[i].hist(df_hist[col], bins=20, color='orange', alpha=0.7)
    ax[i].set(title=col)

# Usuwanie pustych subplotów
for j in range(i + 1, len(ax)):
    fig.delaxes(ax[j])
plt.tight_layout()
plt.show()

In [ ]:
# Tworzymy macierz wykresu za pomocą Seaborn
sns.pairplot(df_hist, diag_kind='kde', plot_kws={'color': 'orange'}, diag_kws={'color': 'orange'})
plt.show()

In [ ]:
# Dzieli dane na model testowy i treningowy
X_train, X_test, y_train, y_test = train_test_split(df.values, target.values, test_size=0.2, random_state=42)

In [ ]:
# konwertujemy wartosc na kalse binarną
median_rent = target.median()[0]
y_train_class = (y_train >= median_rent).astype(int).ravel()
y_test_class = (y_test >= median_rent).astype(int).ravel()

In [ ]:
# Definiujemy architekturę sieci neuronowej za pomocą elastycznych warstw
class WineQualityNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super(WineQualityNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            prev_dim = hidden_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
# Definiujemy wymiary wejściowe i wyjściowe
input_dim = X_train.shape[1]
output_dim = 2

In [ ]:
# Tworzymy siec neuronową
net = NeuralNetClassifier(
    module=WineQualityNN,
    max_epochs=20,
    lr=0.1,
    optimizer=optim.Adam,
    criterion=nn.CrossEntropyLoss,
    iterator_train__shuffle=True,
    verbose=0,
    module__input_dim=input_dim,
    module__output_dim=output_dim
)

In [ ]:
# Podajemy parametry dla GridSearcha
params = {
    'lr': [0.01,0.2,0.4],
    'max_epochs': [10,20,30],
    'module__hidden_dims': [
        [32],[64],[32,32]
    ],
    'optimizer__weight_decay': [0, 1e-4, 1e-2]
}

In [ ]:
# Wykonujemy GridSearch
gs = GridSearchCV(net, params, refit=True, cv=3, scoring='accuracy')
gs.fit(X_train.astype(np.float32), y_train_class)

In [ ]:
# Sprawdzamy dla jakich parametróww mamy najlepsza walidacje krzyżową
print("Best parameters found: ", gs.best_params_)
print("Best cross-validation accuracy: ", gs.best_score_)

In [ ]:
# Sprawdzamy wszytskie rezultaty
results = pd.DataFrame(gs.cv_results_)
print(results)

In [ ]:
# Wyszukujemy najlepszy model na zestawie testowym
best_model = gs.best_estimator_
test_accuracy = best_model.score(X_test.astype(np.float32), y_test_class)
print("Test set accuracy: ", test_accuracy)

In [ ]:
# Wykres i uwzgledneinie kolorów
colors = ['red', 'green', 'blue']

# Tworzenie figury i subplots
fig, axs = plt.subplots(3, 1, figsize=(8, 18))

# Wykres Validation Accuracy vs Learning Rate
axs[0].set_title('Validation Accuracy vs Learning Rate')
for i, max_epoch in enumerate(params['max_epochs']):
    subset = results[results['param_max_epochs'] == max_epoch]
    axs[0].plot(subset['param_lr'], subset['mean_test_score'], label=f'epochs={max_epoch}', color=colors[i])
axs[0].set_xlabel('Learning Rate')
axs[0].set_ylabel('Mean Validation Accuracy')
axs[0].legend()

# Wykres Validation Accuracy vs Number of Epochs
axs[1].set_title('Validation Accuracy vs Number of Epochs')
for i, lr in enumerate(params['lr']):
    subset = results[results['param_lr'] == lr]
    axs[1].plot(subset['param_max_epochs'], subset['mean_test_score'], label=f'lr={lr}', color=colors[i])
axs[1].set_xlabel('Number of Epochs')
axs[1].set_ylabel('Mean Validation Accuracy')
axs[1].legend()

# Wykres Validation Accuracy vs Hidden Layer Configuration
axs[2].set_title('Validation Accuracy vs Hidden Layer Configuration')
hidden_dims_str = [str(hd) for hd in params['module__hidden_dims']]
hidden_dims_mapping = {str(hd): i for i, hd in enumerate(params['module__hidden_dims'])}
results['hidden_dims_str'] = results['param_module__hidden_dims'].astype(str)
for i, weight_decay in enumerate(params['optimizer__weight_decay']):
    subset = results[results['param_optimizer__weight_decay'] == weight_decay]
    axs[2].plot([hidden_dims_mapping[hd] for hd in subset['hidden_dims_str']], subset['mean_test_score'], label=f'wd={weight_decay}', color=colors[i])
axs[2].set_xlabel('Hidden Layer Configuration')
axs[2].set_xticks(range(len(hidden_dims_str)))
axs[2].set_xticklabels(hidden_dims_str, rotation=90)
axs[2].set_ylabel('Mean Validation Accuracy')
axs[2].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Określamy ważność funkcji przy użyciu ważności permutacji
perm_importance = permutation_importance(best_model, X_test.astype(np.float32), y_test_class, n_repeats=10, random_state=42)
feature_importance = perm_importance.importances_mean
feature_names = df.columns

# Wyznaczenie cech fabuły
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, align='center', color='orange')
plt.yticks(range(len(feature_importance)), feature_names)
plt.xlabel('Permutation Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

In [ ]:
# Macierz pomyłek
y_pred = best_model.predict(X_test.astype(np.float32))
cm = confusion_matrix(y_test_class, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Oranges)
plt.title('Confusion Matrix')
plt.show()

In [ ]:
# Krzywa ROC
y_proba = best_model.predict_proba(X_test.astype(np.float32))
fpr, tpr, _ = roc_curve(y_test_class, y_proba[:, 1])
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Raport klasyfikacji
print(classification_report(y_test_class, y_pred))

In [ ]:
# Określamy straty w treningu na przestrzeni epok dla najlepszego modelu
plt.figure(figsize=(8, 5))
plt.plot(best_model.history[:, 'train_loss'], label='Train Loss', color='orange')
plt.xlabel('Epochs')
plt.ylabel('Training Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()